# **LATS Implementation without External Web Search Tools**

# 1. Setting Up the Environment

In [ ]:
from __future__ import annotations
import getpass
import os
import json
import math
from collections import deque
from typing import Optional, Literal
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import (
    JsonOutputToolsParser,
    PydanticToolsParser,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import chain as as_runnable
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableConfig
from IPython.display import Image, display, Markdown
from collections import defaultdict

llm = ChatOpenAI(model="gpt-4", openai_api_key="") #enter open-Ai key here

# 2. Class Declarations of Node, Tree State and Reflection

In [ ]:
class Node:
    def __init__(
        self,
        messages: list[BaseMessage],
        reflection: Reflection,
        parent: Optional[Node] = None,
    ):
        self.messages = messages
        self.parent = parent
        self.children = []
        self.value = 0
        self.visits = 0
        self.reflection = reflection
        self.depth = parent.depth + 1 if parent is not None else 1
        self._is_solved = reflection.found_solution if reflection else False
        if self._is_solved:
            self._mark_tree_as_solved()
        self.backpropagate(reflection.normalized_score)
        print(f"Created node : {self}")

    def __repr__(self) -> str:
        return (
            f"<Node value={self.value:.2f}, visits={self.visits},"
            f" Response={self.messages[-1].content[:50] if self.messages else 'No messages'}...,"
            f" Reflection={self.reflection.reflections[:50] if self.reflection else 'No reflection'}...,"
            f" is_solved={self._is_solved}, depth={self.depth}>"
        )

    @property
    def is_solved(self):
        return self._is_solved

    @property
    def is_terminal(self):
        return not self.children

    @property
    def best_child(self):
        if not self.children:
            return None
        all_nodes = self._get_all_children()
        return max(all_nodes, key=lambda child: child.upper_confidence_bound())

    @property
    def best_child_score(self):
        if not self.children:
            return None
        return max(self.children, key=lambda child: int(child.is_solved) * child.value)

    @property
    def height(self) -> int:
        if self.children:
            return 1 + max([child.height for child in self.children])
        return 1

    def upper_confidence_bound(self, exploration_weight=1.0):
        if self.parent is None:
            raise ValueError("Cannot obtain UCT from root node")
        if self.visits == 0:
            return float('inf')
        average_reward = self.value / self.visits
        exploration_term = math.sqrt(math.log(self.parent.visits) / self.visits)
        return average_reward + exploration_weight * exploration_term

    def backpropagate(self, reward: float):
        node = self
        while node:
            node.visits += 1
            node.value = (node.value * (node.visits - 1) + reward) / node.visits
            node = node.parent

    def get_messages(self, include_reflections: bool = True):
        if include_reflections:
            return self.messages + [self.reflection.as_message()]
        return self.messages

    def get_trajectory(self, include_reflections: bool = True) -> list[BaseMessage]:
        messages = []
        node = self
        while node:
            messages.extend(
                node.get_messages(include_reflections=include_reflections)[::-1]
            )
            node = node.parent
        return messages[::-1]

    def _get_all_children(self):
        all_nodes = []
        nodes = deque([self])
        while nodes:
            node = nodes.popleft()
            all_nodes.extend(node.children)
            nodes.extend(node.children)
        return all_nodes

    def get_best_solution(self):
        all_nodes = [self] + self._get_all_children()
        best_node = max(
            all_nodes,
            key=lambda node: int(node.is_terminal and node.is_solved) * node.value,
        )
        return best_node

    def _mark_tree_as_solved(self):
        parent = self.parent
        while parent:
            parent._is_solved = True
            parent = parent.parent
#---------------------------------------------------------------------------------------------------------------------------

class Reflection(BaseModel):
    reflections: str = Field(
        description="The critique and reflections on the sufficiency, superfluency,"
        " and general quality of the response."
    )
    score: int = Field(
        description="Score from 0-10 on the quality of the candidate response.",
        ge=0,
        le=10,
    )

    found_solution: bool = Field(
        description="Whether the response has fully and perfectly solved the question or task.\
         This should never be true unless an except exceptional answer is generated")

    def as_message(self):
        return HumanMessage(
            content=f"Reasoning: {self.reflections}\nScore: {self.score}"
        )

    @property
    def normalized_score(self) -> float:
        return self.score / 10.0


#---------------------------------------------------------------------------------------------------------------------------

class TreeState(TypedDict):
    root: Node
    input: str


# 3. Reflection

In [ ]:

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Reflect and grade the assistant response to the user question below. \
             Be highly critical in response and dont be satisfied easily\
             Check for following critera 1. Relevance to the question 2. Factual Correctness 3. Quality of text",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="candidate"),
    ]
)

reflection_llm_chain = (
    prompt
    | llm.bind_tools(tools=[Reflection], tool_choice="Reflection").with_config(
        run_name="Reflection"
    )
    | PydanticToolsParser(tools=[Reflection])
)
@as_runnable
def reflection_chain(inputs) -> Reflection:
    tool_choices = reflection_llm_chain.invoke(inputs)
    reflection = tool_choices[0]
    if not isinstance(inputs["candidate"][-1], AIMessage):
        reflection.found_solution = False
    print(f"Generated reflection: {reflection} \n")
    return reflection


# 4. Initial Response with Reflection

In [ ]:

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant. Your job is to answer user question in an accurate and concise manner ",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

initial_answer_chain = prompt_template | llm.with_config(run_name="GenerateInitialCandidate")

parser = JsonOutputToolsParser(return_id=True)

def generate_initial_response(state: TreeState) -> dict:
    print("Generating initial response")
    res = initial_answer_chain.invoke({"input": state["input"]})
    output_messages = [res]
    content = res.content
    display(Markdown(content))
    # print(f"Initial response: {res.content[:100]}...")
    reflection = reflection_chain.invoke(
        {"input": state["input"], "candidate": output_messages}
    )
    # print(f"\nInitial reflection: {reflection} \n ")
    root = Node(output_messages, reflection=reflection)
    print(f"Initial root node created: {root}")
    return {
        **state,
        "root": root,
    }


# 5. Tree Expansion

In [ ]:

def generate_candidates(messages: ChatPromptValue, config: RunnableConfig):
    n = config["configurable"].get("N", 5)
    print(f"Generating {n} candidates")
    chat_result = llm.generate(
        [messages.to_messages()],
        n=n,
        callbacks=config["callbacks"],
        run_name="GenerateCandidates"
    )
    return [gen.message for gen in chat_result.generations[0]]

expansion_chain = prompt_template | generate_candidates

def expand(state: TreeState, config: RunnableConfig) -> dict:
    print("Expanding tree \n")
    root = state["root"]
    best_candidate: Node = root.best_child if root.children else root
    print(f"Best candidate for expansion : {best_candidate} \n")
    messages = best_candidate.get_trajectory()

    new_candidates = expansion_chain.invoke(
        {"input": state["input"], "messages": messages}, config
    )
    print(f"Generated {len(new_candidates)} new candidates \n")

    output_messages = [[candidate] for candidate in new_candidates]

    reflections = reflection_chain.batch(
        [{"input": state["input"], "candidate": msges} for msges in output_messages],
        config,
    )

    child_nodes = [
        Node(cand, parent=best_candidate, reflection=reflection)
        for cand, reflection in zip(output_messages, reflections)
    ]
    best_candidate.children.extend(child_nodes)
    print(f"\n Added {len(child_nodes)} child nodes to the tree \n")

    return state


def should_loop(state: TreeState) -> Literal["expand", "__end__"]:
    root = state["root"]
    print(f"Checking if should loop again. Root height: {root.height}, Solution Found: {root.is_solved} \n")
    if root.is_solved:
        print("Root is solved. Ending search. \n")
        return END
    if root.height > 5:
        print("Max height reached. Ending search. \n ")
        return END
    print("Continuing to expand. \n")
    return "expand"


# 6. Build Graph

In [ ]:

builder = StateGraph(TreeState)
builder.add_node("start", generate_initial_response)
builder.add_node("expand", expand)
builder.add_edge(START, "start")

builder.add_conditional_edges(
    "start",
    should_loop,
)
builder.add_conditional_edges(
    "expand",
    should_loop,
)

graph = builder.compile()

# 7. Tree Search for best answer

In [ ]:
def print_tree(node, level=0):
    print("  " * level + str(node))
    for child in node.children:
        print_tree(child, level + 1)

def run_tree_search(question):
    print(f"Starting tree search for question")
    last_step = None
    for step in graph.stream({"input": question}):
        last_step = step
        step_name, step_state = next(iter(step.items()))
        print(f"Step: {step_name}")
        print(f"Tree height: {step_state['root'].height}")
        print("--------------------------------------------------------")

    if "expand" in last_step:
        solution_node = last_step["expand"]["root"].get_best_solution()
        best_trajectory = solution_node.get_trajectory(include_reflections=False)
        print("Best solution found:")
        # print(best_trajectory[-1].content)
        content = best_trajectory[-1].content
        display(Markdown(content))
    else:
        print("Tree expansion ended \n ")

    print("Final tree structure:")
    print_tree(last_step["start"]["root"] if "start" in last_step else last_step["expand"]["root"])


# 8. Test query

In [ ]:
question = "Generate a table with the average size and weight, as well as the oldest recorded instance for each of the top 5 most common birds."
run_tree_search(question)

Starting tree search for question
Generating initial response


I'm sorry for the inconvenience, but as an AI text-based model, I'm unable to generate tables. However, I can provide the information in text form.

1. House Sparrow
   - Average Size: 16 cm
   - Average Weight: 24-39.5 g
   - Oldest Recorded Instance: 13 years

2. European Starling
   - Average Size: 20 cm
   - Average Weight: 60-100 g
   - Oldest Recorded Instance: 15 years

3. Rock Pigeon
   - Average Size: 32-37 cm
   - Average Weight: 238-380 g
   - Oldest Recorded Instance: 15 years

4. American Robin
   - Average Size: 23-28 cm
   - Average Weight: 72-94 g
   - Oldest Recorded Instance: 14 years

5. Mourning Dove
   - Average Size: 24-33 cm
   - Average Weight: 112-170 g
   - Oldest Recorded Instance: 31 years

Please note that these values are averages and can vary based on individual characteristics and environmental factors. The age of the oldest recorded instance can also vary depending on the source of the information.

Generated reflection: reflections="The assistant's response is relevant to the user's question, providing detailed information in a structured text form since it's unable to generate tables. The assistant provides the average size, weight, and oldest recorded instance of the top 5 most common birds, which is exactly what the user requested. The response is factually correct to the best of my knowledge. The assistant also explains that these values are averages and can vary, which shows attention to detail and understanding of the subject matter. The quality of text is high; it's clear, easy to understand, and well-structured." score=8 found_solution=False 

Created node : <Node value=0.80, visits=1, Response=I'm sorry for the inconvenience, but as an AI text..., Reflection=The assistant's response is relevant to the user's..., is_solved=False, depth=1>
Initial root node created: <Node value=0.80, visits=1, Response=I'm sorry for the inconvenience, but as an AI text..., Reflection=The a

I'm sorry for the inconvenience, but as an AI text-based model, I'm unable to generate tables. However, I can provide the information in text form.

1. House Sparrow
   - Average Size: 16 cm
   - Average Weight: 24-39.5 g
   - Oldest Recorded Instance: 13 years

2. European Starling
   - Average Size: 20 cm
   - Average Weight: 60-100 g
   - Oldest Recorded Instance: 15 years

3. Rock Pigeon
   - Average Size: 32-37 cm
   - Average Weight: 238-380 g
   - Oldest Recorded Instance: 15 years

4. American Robin
   - Average Size: 23-28 cm
   - Average Weight: 72-94 g
   - Oldest Recorded Instance: 14 years

5. Mourning Dove
   - Average Size: 24-33 cm
   - Average Weight: 112-170 g
   - Oldest Recorded Instance: 31 years

Please note that these values are averages and can vary based on individual characteristics and environmental factors. The age of the oldest recorded instance can also vary depending on the source of the information.

Final tree structure:
<Node value=0.61, visits=21, Response=I'm sorry for the inconvenience, but as an AI text..., Reflection=The assistant's response is relevant to the user's..., is_solved=True, depth=1>
  <Node value=0.74, visits=16, Response=I'm sorry for the inconvenience, but as an AI text..., Reflection=Assistant's response is relevant to the user's que..., is_solved=True, depth=2>
    <Node value=0.70, visits=6, Response=I'm sorry for the inconvenience, but as an AI text..., Reflection=The assistant's response was both relevant and fac..., is_solved=False, depth=3>
      <Node value=0.70, visits=1, Response=I'm sorry for the inconvenience, but as an AI text..., Reflection=The assistant's response is highly relevant to the..., is_solved=False, depth=4>
      <Node value=0.70, visits=1, Response=I'm sorry for the inconvenience, but as an AI text..., Reflection=The assistant's response was relevant to the quest..., is_solved=False, depth=4>
      <Node value=0.60, visits=1, Respon